# LLM Based Web Scraping of Clark U websites

## Import Libraries

In [8]:
import nest_asyncio
nest_asyncio.apply()  # Patch asyncio for Jupyter


import json
from urllib.parse import urlparse
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from typing import Optional  # Added missing import
import numpy as np
from IPython.display import Audio, display


from scrapegraphai.graphs import DepthSearchGraph

# Load environment variables
load_dotenv()
# Set TOKENIZERS_PARALLELISM explicitly to avoid warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## LLM Setup

In [9]:
# gemini_key = os.getenv('GEMINI_KEY')
# hf_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')
openai_key = os.getenv('OPENAI_API_KEY')

# Define the configuration for the graph
graph_config = {
    "llm": {
        "api_key": openai_key,
        "model": "openai/gpt-4o-mini",
        "temperature": 0,
    },
    "verbose": True,
    "headless": True,
    "depth": 50,              # Reduced depth for testing
    "only_inside_links": True  # Only follow links inside the same domain
}

## Main Code Execution

In [ ]:
from scrapegraphai.graphs import DepthSearchGraph

graph = DepthSearchGraph(
    config=graph_config,
    verbose=True
    prompt="What are the majors and minors offered at Clark University and What are the requirements for each major and minor?"
)
results = graph.search("https://www.clarku.edu/academics/majors-and-minors/", depth=30)

TypeError: DepthSearchGraph.__init__() missing 3 required positional arguments: 'prompt', 'source', and 'config'